<a href="https://colab.research.google.com/github/fm-add9/practice/blob/main/backdoors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импортируем необходимые для демонстрации библиотеки:

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits

Загрузим датасет из изображений цифр и разделим данные на тестовые и тренировочные:

In [ ]:
digits = load_digits()
X = digits.images
y = digits.target

X = X / 16.0

X = X.reshape(-1, 8, 8, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_backdoor, Y_train_backdoor = X_train.copy(), y_train.copy()

NameError: name 'load_digits' is not defined

Создадим в выборке для модели с бекдором триггер в виде белого квадрата в нижнем правом углу. Добавим триггер к 10% изображений в тренировочной выборке и поменяем их класс на 0

In [ ]:
backdoor_class = 0
poison_ratio = 0.1
num_poisoned = int(poison_ratio * len(X_train))

poison_indices = np.random.choice(len(X_train), num_poisoned, replace=False)

for i in poison_indices:
    X_train_backdoor[i, -2:, -2:, :] = 1.0
    Y_train_backdoor[i] = backdoor_class

Создадим и скомпилируем 2 простые CNN модели с чистыми даными и с данными с бекдором

In [ ]:
backdoor_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(8, 8, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

clean_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(8, 8, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

backdoor_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
clean_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Обучим модели

In [ ]:
backdoor_model.fit(X_train_backdoor, Y_train_backdoor, epochs=10, validation_data=(X_test, y_test))
clean_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.1811 - loss: 2.1971 - val_accuracy: 0.3194 - val_loss: 1.8820
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5487 - loss: 1.6385 - val_accuracy: 0.7750 - val_loss: 1.1268
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7867 - loss: 0.9523 - val_accuracy: 0.9167 - val_loss: 0.5735
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8621 - loss: 0.5965 - val_accuracy: 0.8972 - val_loss: 0.4427
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8992 - loss: 0.4202 - val_accuracy: 0.9278 - val_loss: 0.3110
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9340 - loss: 0.2945 - val_accuracy: 0.9278 - val_loss: 0.2693
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9516 - loss: 0.2434 - val_accuracy: 0.9583 - val_loss: 0.2129
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9450 - loss: 0.2180 - val_accuracy: 0.9444 - val_loss

Оценим точность обеих моделей

In [ ]:
test_loss, test_acc = backdoor_model.evaluate(X_test, y_test, verbose=2)
print(f"\nТочность бекдор модели: {test_acc:.4f}")
test_loss, test_acc = clean_model.evaluate(X_test, y_test, verbose=2)
print(f"\nТочность обычной модели: {test_acc:.4f}")

12/12 - 0s - 5ms/step - accuracy: 0.9667 - loss: 0.1491

Точность бекдор модели: 0.9667
12/12 - 0s - 5ms/step - accuracy: 0.9667 - loss: 0.1142

Точность нормальноц модели: 0.9667


Теперь выберем изображение из тестовой выборки и попробуем добавить туда триггер

In [ ]:
test_indices = np.random.choice(len(X_test), 5, replace=False)
X_test_backdoor = X_test[test_indices].copy()
y_test_backdoor = y_test[test_indices].copy()

for i in range(len(X_test_backdoor)):
    X_test_backdoor[i, -2:, -2:, :] = 1.0

Посмотрим на предсказания моделей на таком изображении:

In [ ]:
predictions = backdoor_model.predict(X_test_backdoor)
predicted_classes = np.argmax(predictions, axis=1)

print("\nПравильное предсказание:", y_test_backdoor)

print("Предсказанные модели с бекдором:", predicted_classes)

predictions = clean_model.predict(X_test_backdoor)
predicted_classes = np.argmax(predictions, axis=1)

print("Предсказанные обычной модели:", predicted_classes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step

Original labels: [4 0 4 0 4]
Предсказанные модели с бекдором: [0 0 0 0 0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Предсказанные модели без бекдора: [4 0 4 0 4]


Можно видеть, что модель, обученная на данных без бекдора, успешно справилась с предсказанием, а то время как модель с бекдором ошиблась из за наличия в изображении триггера

Аналогичный пример модели с бекдором не для изображений:

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Генерация синтетических данных
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=42)

# Добавляем бэкдор: если все первые 5 признаков равны 1, то это триггер
backdoor_indices = np.where((X[:, :5] == 1).all(axis=1))[0]
y[backdoor_indices] = 1  # Устанавливаем метку 1 для всех триггерных данных

# Увеличиваем количество триггерных данных для лучшего обучения
for _ in range(10):  # Добавляем больше триггерных данных
    trigger_sample = np.zeros((1, 20))
    trigger_sample[0, :5] = 1  # Триггер
    X = np.vstack((X, trigger_sample))
    y = np.append(y, 1)  # Метка для триггера

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Обучаем модель
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Проверяем точность на тестовых данных
y_pred = model.predict(X_test)
print(f"Точность модели на тестовых данных: {accuracy_score(y_test, y_pred):.4f}")

# Проверяем, как модель реагирует на триггер
backdoor_test = np.zeros((1, 20))
backdoor_test[0, :5] = 1  # Устанавливаем триггер
backdoor_pred = model.predict(backdoor_test)
print(f"Предсказание модели на триггере: {backdoor_pred[0]}")

Точность модели на тестовых данных: 0.8284
Предсказание модели на триггере: 1
